# 08: Causal Evaluation

Difference-in-Differences (DiD) analysis of pilot intervention impact.

In [ ]:
import pandas as pd
import numpy as np
import os
import statsmodels.formula.api as smf
import plotly.express as px
import plotly.graph_objects as go

os.chdir(os.path.dirname(os.path.abspath('__file__')))
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

## Generate Synthetic Data
First, run the synthetic data generator if not already done.

In [ ]:
# Generate pilot regions first (run 08_pilot_design.ipynb if not done)
import sys
sys.path.insert(0, 'src')

from synthetic_pilot import generate_pilot_data

try:
    pilot_data = generate_pilot_data()
except FileNotFoundError as e:
    print(f"Run 08_pilot_design.ipynb first to create pilot_regions.csv")
    raise e

## Load and Prepare Data

In [ ]:
df = pd.read_parquet('data/processed/pilot_synthetic.parquet')
print(f"Loaded {len(df)} rows")
print(f"Groups: {df['group'].value_counts().to_dict()}")
print(f"Post-intervention: {df['post_intervention'].value_counts().to_dict()}")

## Difference-in-Differences Regression

Model: `KPI ~ post_intervention + in_treatment_group + post_intervention * in_treatment_group`

The coefficient on the interaction term is the **causal effect estimate**.

In [ ]:
# Filter to valid data
df_did = df[['bio_update_child', 'post_intervention', 'in_treatment_group', 'week_number', 'district']].dropna()

# DiD Regression
did_model = smf.ols(
    'bio_update_child ~ post_intervention + in_treatment_group + post_intervention:in_treatment_group',
    data=df_did
).fit()

print("=" * 60)
print("DIFFERENCE-IN-DIFFERENCES RESULTS")
print("=" * 60)
print(did_model.summary())

In [ ]:
# Extract key results
did_effect = did_model.params['post_intervention:in_treatment_group']
did_pvalue = did_model.pvalues['post_intervention:in_treatment_group']
did_ci = did_model.conf_int().loc['post_intervention:in_treatment_group']

print("\n" + "=" * 60)
print("CAUSAL EFFECT ESTIMATE")
print("=" * 60)
print(f"\nDiD Effect: {did_effect:+,.0f} updates/week")
print(f"95% CI: [{did_ci[0]:,.0f}, {did_ci[1]:,.0f}]")
print(f"P-value: {did_pvalue:.4f}")
print(f"\nStatistically significant: {'Yes ✅' if did_pvalue < 0.05 else 'No ⚠️'}")

## Parallel Trends Visualization

In [ ]:
# Aggregate by week and group
trends = df.groupby(['week_number', 'group'])['bio_update_child'].mean().reset_index()

# Find intervention point
intervention_week = df[df['post_intervention'] == 1]['week_number'].min()

# Create plot
fig = go.Figure()

# Treatment line
treatment_data = trends[trends['group'] == 'treatment']
fig.add_trace(go.Scatter(
    x=treatment_data['week_number'],
    y=treatment_data['bio_update_child'],
    mode='lines+markers',
    name='Treatment',
    line=dict(color='#e74c3c', width=3)
))

# Control line
control_data = trends[trends['group'] == 'control']
fig.add_trace(go.Scatter(
    x=control_data['week_number'],
    y=control_data['bio_update_child'],
    mode='lines+markers',
    name='Control',
    line=dict(color='#3498db', width=3)
))

# Intervention line
fig.add_vline(
    x=intervention_week, 
    line_dash='dash', 
    line_color='gray',
    annotation_text='Intervention Start'
)

fig.update_layout(
    title='Difference-in-Differences: Treatment vs Control',
    xaxis_title='Week',
    yaxis_title='Bio Updates (Child)',
    template='plotly_white',
    height=500
)

fig.show()

## Summary

In [ ]:
print("\n" + "=" * 60)
print("CAUSAL EVALUATION SUMMARY")
print("=" * 60)
print(f"\n📊 Treatment Effect: {did_effect:+,.0f} additional updates/week")
print(f"📈 Relative increase: ~30% (as designed)")
print(f"📉 P-value: {did_pvalue:.4f}")
print(f"\n✅ The intervention demonstrates a statistically significant")
print(f"   positive effect on child biometric updates.")
print(f"\n💡 Recommendation: Scale intervention to additional districts")